# Self-Driving Car Engineer Nanodegree


## Project: **Advanced Lane Finding** 
The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
---

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
%matplotlib inline
#%matplotlib qt
%qtconsole

## Classes

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = 0 
        #distance in meters of vehicle center from the line
        self.line_base_pos = 0 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None 
    def getAvgFit(self):
        if self.best_fit == None:
            self.best_fit = self.current_fit
        #Take a weighted average of the new and old coeffecicients. Weight the old ones 5 times as much as the new
        self.best_fit = (self.best_fit*5 + self.current_fit)/6
        return self.best_fit

## Helper Functions

In [ ]:
##################################################
#**Helper Functions for Camera Calibration**
#####################################################
# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

#################################################################
#**Helper functions for color and gradient thresholds**
#################################################################
#Takes in threshold for HLS color space and for sobel in the x direction 
#Idea: run color threshold and then run sobel on each color threshold and "or" them!
def sobelx_thresh(img,sx_thresh=(15, 100),sobel_kernel=5):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0,ksize=sobel_kernel) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    return sxbinary

def sobel_grad_thresh(img,  s_thresh=(150, 255),l_thresh=(199, 255),r_thresh=(215, 255),sx_thresh=(15, 100),sobel_kernel=5):
    img = np.copy(img)
    r_channel = img[:,:,0]
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    sobel_s = sobelx_thresh(s_binary,sx_thresh,sobel_kernel)
    #Aaron added l to look at light space
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    sobel_l = sobelx_thresh(l_binary,sx_thresh,sobel_kernel)
    #Aaron added R to look at whites better
    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel >= r_thresh[0]) & (r_channel <= r_thresh[1])] = 1 #made zero to get rid of this
    sobel_r = sobelx_thresh(r_binary,sx_thresh,sobel_kernel)

    #Now combine the sobel binarys
    combined = np.zeros_like(sobel_r)
    #combined[(sobel_s==1) | (sobel_l==1) | (sobel_r==1)]=1 #or everything
    combined[((s_binary==1)&(r_binary==1)) | (sobel_l==1)]=1
    combined= sobelx_thresh(combined,sx_thresh,sobel_kernel)
    
    
    # Stack each channel
    #color_binary = np.dstack(( np.zeros_like(sxbinary), l_binary, r_binary)) * 255
    color_binary_beforeSobel = np.dstack((r_binary, s_binary, l_binary)) * 255
    color_binary_withSobel = np.dstack((sobel_r, sobel_s, sobel_l)) * 255
    return (color_binary_beforeSobel, color_binary_withSobel, combined)

###################################################
#** Helper functions for Perspective Transform**
###################################################
def warp(img, topRight=[682,445],bottomRight =[1054,677],bottomLeft=[250,680],topLeft=[598,445] ):
    #Define calibration box in source (origional) and destination (desired or warped) coordinates
    img_size = (img.shape[1],img.shape[0])
    
    #Four source coordinates:
    src = np.float32(
        [topRight,#Top Right
         bottomRight,#Bottom Right
         bottomLeft,#Bottom left
         topLeft])#Top left


    #Four desired coordinates (define a rectangle)
    dst = np.float32(
        [[900,0],
         [900,720],
         [250,720],
         [250,0]])
    
    #compute the perspective transform, M
    M = cv2.getPerspectiveTransform(src,dst)
    
    #Could compute the inverse also by swapping the input parameters
    Minv = cv2.getPerspectiveTransform(dst,src)
    
    #Create warped image - uses linear interpolation
    warped = cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    
    return warped, Minv


####################################################################
# Helper functions for detecting lane pixels and identifying lane boundary
######################################################################

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped) 

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
   
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    
    #Now calculate the radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return out_img, left_curverad, right_curverad, left_fit,right_fit
    
    
#######################################
# Now unwarp image back
#######################################
def unwarp(image,binary_warped, Minv, left_fit, right_fit):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    unwarped_result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
        
    return unwarped_result

#################################################
# Functions for checking if detected lines are ok
# returns a boolean for if we can trust the lines
#################################################
def checkLines(left_fit, right_fit, left_curverad, right_curverad):
    if left_fit[0]<0:
        if right_fit[0]>0:
            return False
    elif left_fit[0]>0:
        if right_fit[0]<0:
            return False
    
    percentDiff_rad = (left_curverad-right_curverad)/max(min(left_curverad,right_curverad),1)*100
    if (percentDiff_rad > 200):
        return False
    
    #If we get here then we can trust the lines:
    return True

#Returns Radius of curvurature and offset
def calcRadAndOffset(binary_warped, left_fit, right_fit, left_curverad, right_curverad):
    avgRad = (left_curverad+right_curverad)/2
    maxy = binary_warped.shape[0]-1
    middle = binary_warped.shape[1]//2
    left_base = left_fit[0]*maxy**2+left_fit[1]*maxy+left_fit[2]
    right_base = right_fit[0]*maxy**2+right_fit[1]*maxy+right_fit[2]
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    offset = ((left_base+right_base)/2-middle)*xm_per_pix #negative is left of center
    return (avgRad,offset)

def getOffsetStr(offset):
    if offset>0:
        offsetStr = "The vehicle is "+str(round(offset,2))+ " (m) right of center."
    else:
        offsetStr = "The vehicle is "+str(abs(round(offset,2)))+ " (m) left of center."
    return offsetStr

def writeText2Image(img, radStr, offStr):
    font = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (350,100)
    fontScale = 1
    fontColor = (255,255,255)
    lineType = 2
    cv2.putText(img,radStr, bottomLeftCornerOfText, font, fontScale,fontColor,lineType)
    # Write the offset text
    bottomLeftCornerOfText = (350,150)
    cv2.putText(img,offStr, bottomLeftCornerOfText, font, fontScale,fontColor,lineType)
    
    
#######################################################
## Helper functions for search from prior polynomial ##
#######################################################
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, right_fit, ploty

#Note: for this fucntion, left_fit and right_fit must be taken from the line class as they are the previous fits
def search_around_poly(binary_warped):
    left_fit = leftLine.current_fit # Aaron Note: probably want to make this average not current *****************************
    right_fit = rightLine.current_fit # Aaron Note: probably want to make this average not current ****************************
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fit, right_fit, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    

    ## Visualization ##
    #I deleted code for this. Maybe put back for report. Would need to change fit_poly to get to work to return fitx    
    
    return (left_fit, right_fit, ploty)

def calcRad(left_fit, right_fit, ploty):
    #Now calculate the radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return (left_curverad, right_curverad)

## Camera Calibration



### Reading in a chessboard image

In [ ]:
#Reading in a chessboard image example
img = mpimg.imread('camera_cal/calibration2.jpg')
#printing out some stats and plotting
print('This image is:', type(img), 'with dimensions:', img.shape)
plt.imshow(img)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')



In [ ]:
# Read in and make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Arrays to store object points and image points from all the images
objpoints = [] # 3D points in real world space (range from [0,0,0] to [8,5,0] )
imgpoints = [] # 2D points in image plane

#Prepare object points, like (0,0,0), (1,0,0)...(8,5,0)
objp =np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) #x,y coordinates

for fname in images:
    #Read in each image
    img = mpimg.imread(fname) #matplotlb.image.imread Gives an RGB image. #cv2.imread() gives BGR image

    #Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    

    #Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    #if the corners are found, add oject points, image points
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)

        #draw and display the corners
        img = cv2.drawChessboardCorners(img,(9,6), corners, ret)
        #plt.imshow(img)
        #plt.title("Chessboard Corners on Raw Image")
        #plt.savefig('output_images/ChessCornersRaw')
        #plt.show()

# Read in the chess board image and test the undistort
img = mpimg.imread('camera_cal/calibration2.jpg')
#Use the collection of object points and image points from all our chessboard pics to calibrate the camera
undistorted = cal_undistort(img, objpoints, imgpoints)
#Plot the origional and undistorted image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
#plt.savefig('output_images/OrigionalVsUndistortedImage')




## Color Transform with Thresholding and Gradients

In [ ]:
#Step 1- distortion correction
#Now read in a road test image and test the undistort
img = mpimg.imread('test_images/test2.jpg')
#Use the collection of object points and image points from all our chessboard pics to calibrate the camera
undistorted = cal_undistort(img, objpoints, imgpoints)
''' #Uncomment this section to plot for report
#Plot the origional and undistorted image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
#plt.savefig('output_images/OrigionalVsUndistortedImage_road')
plt.show()
'''

#Step 2- Combined Colo and gradient  threshold for lane detection
#In output, green = Sobel and Blue = HLS -S thresh (150,255), Sx=(15,100), L-threh=(199,255),r-thresh=(205, 255), kernel =5
resultBeforeSobel, resultWithSobel,combinedSobel = sobel_grad_thresh(undistorted, s_thresh=(150, 255),l_thresh=(215, 255),r_thresh=(205, 255), sx_thresh=(35, 100),sobel_kernel=5)
# Plot the result
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
f.tight_layout()

ax1.imshow(undistorted)
ax1.set_title('Original Image', fontsize=40)

ax2.imshow(resultBeforeSobel)
ax2.set_title('R,S, L Thresh Only', fontsize=40)

ax3.imshow(resultWithSobel)
ax3.set_title('Color Thresh with Sobel', fontsize=40)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.show()
#plt.savefig('output_images/GradThreshAndSobel')

plt.imshow(combinedSobel,cmap='gray')
plt.title('Combined Sobels')
#plt.savefig('output_images/CombinedSobels')
plt.show()

## Perspective Transform (Bird's Eye View)

In [ ]:
#Read in straight lined image
img = mpimg.imread('test_images/straight_lines1.jpg')
#Undistort the image
undistorted = cal_undistort(img, objpoints, imgpoints)
#Pick four source points and four destination points
plt.imshow(undistorted)
#plt.savefig('output_images/UndistortedStraightLines')
#Source points
plt.plot(682,445,'.')#Top Right
plt.plot(1054,677,'.')#Bottom Right
plt.plot(250,677,'.')#Bottom left
plt.plot(598,445,'.')#Top left


In [ ]:
#Now perform the perspective transform and plot

### This part is just for report - maybe add red lines to show perspective transform ### 
'''
#Get perspective transform
warped_im = warp(img)

#Visualize undistortion
f, (ax1, ax2)=plt.subplots(1,2,figsize=(20,10))

ax1.set_title('Source Image')
ax1.imshow(img)
ax2.set_title('Warped Image')
ax2.imshow(warped_im)
'''


##############################################################
# Do the same visualization for the binary thresholded image #
##############################################################
#Get perspective transform
binary_warped, Minv = warp(combinedSobel)

#Visualize undistortion
f, (ax1, ax2)=plt.subplots(1,2,figsize=(20,10))

ax1.set_title('Source Image')
ax1.imshow(combinedSobel,cmap='gray')
ax2.set_title('Warped Image')
ax2.imshow(binary_warped,cmap='gray')

## Detect Lane Pixels to find Lane Boundary

In [ ]:
#Start by implementing sliding windows
out_img, left_curverad, right_curverad, left_fit, right_fit = fit_polynomial(binary_warped)
plt.imshow(out_img)
#print(left_curverad)
#print(right_curverad)
#print(left_fit) #note: if left_fit[0] is opposite sign of right_fit[0] then I should discard the data
#print(right_fit)

#Check if the left and right fits have the same "slope"
trustLines = False
if left_fit[0]<0:
    if right_fit[0]<0:
        trustLines = True
else:
    if right_fit[0]>0:
        trustLines = True
        
if trustLines:
    avgRad = (left_curverad+right_curverad)/2
    print("The radius of curvature is: "+ str(round(avgRad,2)) + " (m)")

    maxy = binary_warped.shape[0]-1
    middle = binary_warped.shape[1]//2
    left_base = left_fit[0]*maxy**2+left_fit[1]*maxy+left_fit[2]
    right_base = right_fit[0]*maxy**2+right_fit[1]*maxy+right_fit[2]
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    offset = ((left_base+right_base)/2-middle)*xm_per_pix
    if offset>0:
        print("The vehicle is "+str(round(offset,2))+ " (m) right of center.")
    else:
        print("The vehicle is "+str(abs(round(offset,2)))+ " (m) left of center.")
        
else:
    print("Cannot trust lines")

## Pipeline

In [ ]:
#initialize classes
leftLine = Line();
rightLine = Line();

def process_image(image):
    #####################################################
    ## Color Transform with Thresholding and Gradients  # 
    #####################################################
    #Step 1- distortion correction
    #Use the collection of object points and image points from all our chessboard pics to calibrate the camera
    undistorted = cal_undistort(image, objpoints, imgpoints)

    #Step 2- Combined Colo and gradient  threshold for lane detection
    #In output, green = Sobel and Blue = HLS -S thresh (150,255), Sx=(15,100), L-threh=(199,255),r-thresh=(205, 255), kernel =5
    resultBeforeSobel, resultWithSobel,combinedSobel = sobel_grad_thresh(undistorted, s_thresh=(150, 255),l_thresh=(215, 255),r_thresh=(205, 255), sx_thresh=(35, 100),sobel_kernel=5)

    ##############################################
    ## Perspective Transform (Bird's Eye View)  ##
    ##############################################
    #Get perspective transform
    binary_warped, Minv = warp(combinedSobel)

    ##############################################
    ## Detect Lane Pixels to find Lane Boundary ##
    ##############################################
    #Start by implementing sliding windows
    if leftLine.detected == False:
        out_img, left_curverad, right_curverad, left_fit, right_fit = fit_polynomial(binary_warped)
    #Otherwise our leftLine was already detected
    else:
        left_fit, right_fit, ploty = search_around_poly(binary_warped)
        left_curverad, right_curverad = calcRad(left_fit, right_fit, ploty)
        avgRadius, offset = calcRadAndOffset(binary_warped, left_fit, right_fit, right_curverad, right_curverad)
    
    
    trustLines = checkLines(left_fit, right_fit, left_curverad, right_curverad)
    if trustLines==True:
        avgRadius, offset = calcRadAndOffset(binary_warped, left_fit, right_fit, left_curverad, right_curverad)
        radiusStr = "Radius of Curvature = " + str(round(avgRadius,2)) + " (m)"
        offsetStr = getOffsetStr(offset)
        leftLine.detected= True
        rightLine.detected = True
        leftLine.current_fit = left_fit
        rightLine.current_fit = right_fit
        avgLeftFit = leftLine.getAvgFit()
        avgRightFit = rightLine.getAvgFit()
        leftLine.radius_of_curvature = avgRadius
        leftLine.line_base_pos = offset
    else:
        radiusStr = "Radius of Curvature = " + str(leftLine.radius_of_curvature) + " (m)"
        offsetStr = getOffsetStr(leftLine.line_base_pos)   
        if leftLine.best_fit == None:
            avgLeftFit = leftLine.current_fit
            avgRightFit = rightLine.current_fit
        else:
            avgLeftFit = leftLine.best_fit
            avgRightFit = rightLine.best_fit
        
    #This is the result with everything except the text
    unwarped_result = unwarp(image,binary_warped, Minv, avgLeftFit, avgRightFit)
    # Write the radius of curvature  and offset Text
    writeText2Image(unwarped_result, radiusStr, offsetStr)
    
    return unwarped_result



## Test on Videos

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
white_output = 'test_videos_output/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))